we will scrap data like post, comments from reddit and make a dataset will be lated used for Natural language processing.

In [4]:
from core.selenium_scraper import SeleniumScraper
from core.soup_scraper import SoupScraper
from core.progress_bar import ProgressBar
from core.sql_access import SqlAccess
import time

In [2]:
pip install selenium

  Using cached selenium-3.141.0-py2.py3-none-any.whl (904 kB)
Note: you may need to restart the kernel to use updated packages.


In [5]:
reddit_home = 'https://www.reddit.com'
slash = '/r/'
subreddit = 'DataScience'
sort_by = '/hot/'
scroll_n_times = 1000
scrape_comments = True
erase_db_first = True

In [7]:
SQL = SqlAccess()
SelScraper = SeleniumScraper()
BSS = SoupScraper(reddit_home,
                  slash,
                  subreddit)

SelScraper.setup_chrome_browser()

In [8]:
# Collect links from subreddit
links = SelScraper.collect_links(page = reddit_home + 
                                        slash + subreddit + sort_by,
                                 scroll_n_times = scroll_n_times)

Opening reddit and scrolling: takes approximately 500.0 seconds
Collected 0 links


In [9]:
# Find the <script> with id='data' for each link
script_data = BSS.get_scripts(urls = links)


Finding <script id="data"> for each link


In [10]:
# Transforms each script with data into a Python dict, returned as [{}, {}...]
BSS.data = SelScraper.reddit_data_to_dict(script_data = script_data)


Making Python dicts out of script data


In [11]:
print('Scraping data...')
progress = ProgressBar(len(links))
for i, current_data in enumerate(BSS.data):
    progress.update()
    
    BSS.get_url_id_and_url_title(BSS.urls[i],
                                 current_data, i)
    BSS.get_title()
    BSS.get_upvote_ratio()
    BSS.get_score()
    BSS.get_posted_time()
    BSS.get_author()
    BSS.get_flairs()
    BSS.get_num_gold()
    BSS.get_category()
    BSS.get_total_num_comments()
    BSS.get_links_from_post()
    BSS.get_main_link()
    BSS.get_text()
    BSS.get_comment_ids()


Scraping data...


In [14]:
print('Scraping data...')
start = time.time()
progress = ProgressBar(len(links))
for i, current_data in enumerate(BSS.data):
    progress.update()
    
    BSS.get_url_id_and_url_title(BSS.urls[i],
                                 current_data, i)
    BSS.get_title()
    BSS.get_upvote_ratio()
    BSS.get_score()
    BSS.get_posted_time()
    BSS.get_author()
    BSS.get_flairs()
    BSS.get_num_gold()
    BSS.get_category()
    BSS.get_total_num_comments()
    BSS.get_links_from_post()
    BSS.get_main_link()
    BSS.get_text()
    BSS.get_comment_ids()

time.sleep(1)
BSS.prepare_data_for_sql(scrape_comments=scrape_comments)

try:
    SQL.create_or_connect_db(erase_first=erase_db_first)
    # [0] = post, [1] = comment, [2] = link
    for i in range(len(BSS.post_data)):
        SQL.insert('post', data = BSS.post_data[i])
        SQL.insert('link', data = BSS.link_data[i])
        
        if scrape_comments:
            SQL.insert('comment', data = BSS.comment_data[i])
except Exception as ex:
    print(ex)
finally:
    SQL.save_changes()

time.sleep(1)
end = time.time()

print(('\nIt took {0} seconds to scrape and store {1} links').format(round(end - start, 1),
                                                                     len(links)))

Scraping data...
Gathering all the scraped data, and scraping ALL comment data (very slow, dependent on number of comments)

It took 2.0 seconds to scrape and store 0 links


dataset is saved into sql database

In [15]:
#if want to enter new data to database
try:
    SQL.create_or_connect_db(erase_first=erase_db_first)
    # [0] = post, [1] = comment, [2] = link
    for i in range(len(BSS.post_data)):
        SQL.insert('post', data = BSS.post_data[i])
        SQL.insert('link', data = BSS.link_data[i])
        
        if scrape_comments:
            SQL.insert('comment', data = BSS.comment_data[i])
except Exception as ex:
    print(ex)
finally:
    SQL.save_changes()

In [16]:
#now lets make dataset
from core.sql_access import SqlAccess
import pandas as pd

In [17]:
#connect to database
SQL = SqlAccess()
SQL.create_or_connect_db()
c = SQL.conn

In [18]:
#retrieve data from database table
all_data = pd.read_sql_query("""
SELECT *
FROM post p 
LEFT JOIN comment c 
    ON p.id = c.post_id
LEFT JOIN link l
	ON p.id = l.post_id;
""", c)

In [20]:
#collect posts and comments data
post = pd.read_sql_query("""
SELECT *
FROM post;
""", c)

comment = pd.read_sql_query("""
SELECT *
FROM comment;
""", c)

In [21]:
#save as csv
all_data.to_csv('data/post_comment_link_data_demo.csv', columns=all_data.columns, index=False)
post.to_csv('data/post_data_demo.csv', columns=post.columns, index=False)
comment.to_csv('data/comment_data_demo.csv', columns=comment.columns, index=False)